In [1]:
class matchrow:
    def __init__(self , row , allnum = False):
        if allnum:
            self.data = [float(row[i]) for i in range(len(row)-1)]
        else:
            self.data = row[0:len(row) - 1]
        
        self.match = int(row[len(row) - 1])

def loadmatch(f , allnum = False):
    rows = []
    
    for line in file(f):
        rows.append(matchrow(line/split(',') , allnum))
    
    return rows

In [2]:
def lineartrain(rows):
    averages = {}
    counts = {}
    
    for row in rows:
        c1 = row.match
        
        averages.setdefault(c1 , [0.0]*(len(row.data)))
        counts.setdefault(c1 , 0)
        
        for i in range(len(row.data)):
            averages[c1][i] += float(row.data[i])
        
        counts[c1]+=1
    
    for c1 , avg in averages.items():
        for i in range(len(avg)):
            avg[i] /= counts[c1]
        
    return averages

In [3]:
def dotporduct(v1 , v2):
    return sum([v1[i]*v2[i] for i in range(len(v1))])

In [4]:
def dpclassify(point , avgs):
    b = (dotporduct(avgs[1] , avgs[1]) - dotporduct(avgs[0] , avgs[0]))/2
    y = dotporduct(point , avgs[0]) - dotporduct(point , avgs[1]) + b
    
    if y>0:
        return 0
    else:
        return 1

In [5]:
def yesno(v):
    if v=='yes':
        return 1
    elif v == 'no':
        return -1
    else:
        return 0

In [6]:
def matchcount(interest1 , interest2):
    l1 = interest1.split(':')
    l2 = interest2.split(':')
    
    x=0
    
    for v in l1:
        if v in l2:
            x+=1
    
    return x

In [7]:
def milesdistance(a1 , a3):
    return 0

In [8]:
#根据经纬度计算欧几里得距离
def milesdistance(a1 , a2):
    lat1 , long1 = getlocation(a1)
    lat2 , long2 = getlocation(a2)
    
    latdif = 69.1*(lat2-lat1)
    longdif = 53.0*(long2 - long1)
    
    return (latdif**2 + longdif**2)**0.5



In [9]:
def loadnumerical():
    oldrows = loadmatch('matchmaker.csv')
    newrows = []
    
    for row in oldrows:
        d = row.data
        data = [float(d[0]) , yesno(d[1]),yesno(d[2]),
                float(d[5]) , yesno(d[6]) , yesno(d[7]),
                matchcount(d[3] , d[8]),
                milesdistance(d[4] , d[9]),
                row.match]
        newrows.append(matchcount(data))
    
    return newrows

In [10]:
def scaledata(rows):
    low = [999999999.0]*len(rows[0].data)
    high = [-999999999.0]*len(rows[0].data)
    
    for row in rows:
        d = row.data
        for i in range(len(d)):
            if d[i]<low[i]:
                low[i] = d[i]
            
            if d[i]>high[i]:
                high[i] = d[i]
            
    
    def scaleinput(d):
        return [(d.data[i]-low[i])/(high[i]-low[i]) 
               for i in range(len(low))]

    newrows = [matchrow(scaleinput(row.data)+[row.match]) 
              for row in rows]
    
    return newrows , scaleinput

In [11]:
#径向基核函数(高斯核函数)
def rbf(v1 , v2 , gamma=20):
    dv = [v1[i] - v2[i] for i in range(len(v1))]
    l = veclength(dv)
    
    return math.e**(-gamma*l)

In [12]:
def nlclassify(point , rows , offset , gamma=10):
    sum0 = 0.0
    sum1 = 0.0
    count0 = 0
    count1 = 0
    
    for row in rows:
        if row.match == 0:
            sum0 += rbf(point , row.data , gamma)
            count0 += 1
        else:
            sum1 += rbf(point , row.data , gamma)
            count1 += 1
    
    y = (1.0/count0)*sum0-(1.0/count1)*sum1 + offset
    
    if y<0:
        return 0
    else:
        return 1
    
def getoffset(rows , gamma = 10):
    l0 = []
    l1 = []
    
    for row in rows:
        if row.match == 0:
            l0.append(row.data)
        else:
            l1.appen(row.data)
    
    sum0 = sum(sum([rbf(v1 , v2 , gamma) for v1 in l0]) for v2 in l0)
    sum1 = sum(sum([rbf(v1 , v2 , gamma) for v1 in l1]) for v2 in l1)
    
    return (1.0/(len(l1)**2))*sum1 - (1.0/(len(l0)**2)) * sum0

In [13]:
from svm import *

In [14]:
prob = svm_problem([1,-1] , [[1,0,1],[-1,0,-1]])

In [18]:
param = svm_parameter(LINEAR , 10)

TypeError: __init__() takes at most 2 arguments (3 given)

In [ ]:
m = svm_model(prob , param)

In [ ]:
#保存模型 导入模型
m.save(test.model)
m = svm_model(test.model)

In [19]:
answers , inputs = [r.match for r in scaledset] , [r.data for r in scaledset]

NameError: name 'scaledset' is not defined

In [ ]:
param = svm_parameter(kernel_type  = RBF)
prob = svm_problem(answers , inputs)
m = svm_model(prob , param)

In [20]:
guesses = cross_validation(prob , param , 4)
guesses

NameError: name 'cross_validation' is not defined

In [ ]:
sum([abs(answers[i] - guesses[i]) for i in range(len(guesses))])